In [1]:
(define nil '())

# 2.3 Symbolic Data

In this section we extend the representational capability of our language by introducing the ability to work with arbitrary symbols as data.

## 2.3.1 Quotation

$(a\ b) \neq (list\ a\ b)$ 

In [2]:
(define a 1)
(define b 2)
(define c 3)

In [3]:
(list a b c )

(1 2 3)

In [4]:
(list 'a 'b c)

(a b 3)

In [5]:
(car '(a b c))

a

In [6]:
(cdr '(a b c))

(b c)

In [7]:
(eval (car '(a b)))

1

In [8]:
(eq? 1 a)

#t

In [9]:
(eq? 1 'a)

#f

In [10]:
; memq:
; arguments - symbol
;           - list
; If the symbol is not contained in the list, then 'memq' returns false

(define (memq item x)
  (cond ((null? x) #f)
        ((eq? item (car x)) x)
        (else (memq item (cdr x)))))

In [11]:
(memq 'apple '(pepper banana prune))

#f

In [12]:
(memq 'apple '(x (apple sauce) y apple pear))

(apple pear)

In [13]:
(memq 1 '(a b c))

#f

In [14]:
(memq 'a '(a b c))

(a b c)

### Exercise 2.53

```lisp
1. (list 'a 'b 'c)
2. (list (list 'george))
3. (cdr '((x1 x2) (y1 y2)))
4. (cadr '((x1 x2) (y1 y2)))
5. (pair? (car '(a short list)))
6. (memq 'red '((red shoes) (blue socks)))
7. (memq 'red '(red shoes blue socks))
```
1. (a b c)
2. ((george))
3. ((y1 y2))
4. (y1 y2)
5. #f
6. #f
7. (red hoes blue socks)

In [15]:
(list 'a 'b 'c)

(a b c)

In [16]:
(list (list 'george))

((george))

In [17]:
(cdr '((x1 x2) (y1 y2)))

((y1 y2))

In [18]:
(cadr '((x1 x2) (y1 y2)))

(y1 y2)

In [19]:
(pair? (car '(a short list)))

#f

In [20]:
(memq 'red '((red shoes) (blue socks)))

#f

In [21]:
(memq 'red '(red shoes blue socks))

(red shoes blue socks)

### Exercise 2.54

In [22]:
(equal? '(this is a list) '(this is a list))

#t

In [23]:
(equal? '(this (is a) list) '(this is a list))

#f

In [24]:
(equal? '(this a is list) '(this is a list))

#f

In [25]:
;answer

(define (my-equal? xs ys)
  (cond ((and (null? xs) (null? ys)) #t)
        ((or (null? xs) (null? ys)) #f)
        ((and (not (pair? (car xs))) (not (pair? (car ys)))) (eq? (car xs) (car ys)))
        ((equal? (car xs) (car ys)) (my-equal? (cdr xs) (cdr ys)))
        (else #f)))

In [26]:
(my-equal? '(this is a list) '(this is a list))

#t

In [27]:
(my-equal? '(this is a list) '(this (is a) list))

#t

In [28]:
(my-equal? '(this a is list) '(this is a list))

#t

In [29]:
(my-equal? '(this (is a) list) '(this (is a) list))

#t

### Exercise 2.55

In [30]:
(car ''abracadabra)

quote

In [31]:
''abracadabra

(quote abracadabra)

In [32]:
(quasiquote abracadabra)

abracadabra

In [33]:
(quasiquote (quasiquote abracadabra))

(quasiquote abracadabra)

In [34]:
(eq? 'abracadabra (quasiquote abracadabra))
; =
;(eq? 'abracadabra (quote abracadabra))

#t

`symbol　は　(quasiquote symbol)　のシンタックスシュガー

## 2.3.2 Example: Symbolic Differentiation

In [35]:
(define (variable? e)
  (symbol? e))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (sum? e)
  (and (pair? e) (eq? (car e) `+)))

(define (addend e)
  (cadr e))

(define (augend e)
  (caddr e))

;(define (make-sum a1 a2)
;  (list `+ a1 a2))

;modify & add a function '=number?'
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list '+ a1 a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))


(define (product? e)
  (and (pair? e) (eq? (car e) `*)))

(define (multiplier e)
  (cadr e))

(define (multiplicand e)
  (caddr e))

;(define (make-product m1 m2)
;  (list `* m1 m2))

; modify
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))

In [36]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        (else (error "unknown expression type: DERIV" "hoge"))))

In [37]:
(deriv '(+ x 3) 'x)

1

In [38]:
(deriv '(* x y) 'x)

y

In [39]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

### Exercise 2.56

In [40]:
(define (exponentiation? v)
  (and (pair? v)
       (eq? '** (car v))))

(define (base v)
  (cadr v))

(define (exponent v)
  (caddr v))

(define (make-exponentiation b e)
  (cond ((=number? e 1) b)
        ((=number? e 0) 1)
        (else (list '** b e))))


In [41]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))




In [42]:
(deriv '(+ 3 (** (+ x 3) 3)) 'x)

(* 3 (** (+ x 3) 2))

In [43]:
(make-exponentiation 'x 0)

1

In [44]:
(make-exponentiation 'x 2)

(** x 2)

### Exercise 2.57

In [45]:
(define (augend e)
  (if (null? (cdddr e))
      (caddr e)
      (cons '+ (cddr e))))


(define (multiplicand e)
  (if (null? (cdddr e))
      (caddr e)
      (cons '* (cddr e))))

;;
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))

In [46]:
(deriv '(* x (* y (+ x 3))) 'x)

(+ (* x y) (* y (+ x 3)))

In [47]:
(deriv '(* x y (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [48]:
(product? (multiplicand '(* x y (+ x 3))))

#t

In [49]:
(multiplicand (multiplicand '(* x y (+ x 3))))

(+ x 3)

In [50]:
(cdddr '(* y (+ x 3)))

()

### Exercise 2.58

In [51]:
; a.

(define (variable? e)
  (symbol? e))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (sum? e)
  (and (pair? e) (eq? (cadr e) `+)))

(define (addend e)
  (car e))

(define (augend e)
  (caddr e))

;(define (make-sum a1 a2)
;  (list  a1 '+ a2))

;modify & add a function '=number?'
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list  a1 '+ a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))


(define (product? e)
  (and (pair? e) (eq? (cadr e) `*)))

(define (multiplier e)
  (car e))

(define (multiplicand e)
  (caddr e))

;(define (make-product m1 m2)
;  (list  m1 '* m2))

; modify
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list  m1 '* m2))))


In [52]:
;;
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))

In [53]:
(deriv '(x + (3 * (x + (y + 2)))) 'x)

4

In [54]:
; b.

(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        (else (error "unknown expression type: DERIV" "error"))))


(define (augend e)
  (if (null? (cdddr e))
      (caddr e)
      (cond ((product? (cddr e)) (cddr e))
            ((sum? (cddr e)) (cddr e)))))


(define (multiplicand e)
  (if (null? (cdddr e))
      (caddr e)
      (cond ((product? (cddr e)) (cddr e))
            ((sum? (cddr e)) (cddr e)))))


In [55]:
(deriv '(x + 3 * (x + y + 2)) 'x)

4

In [56]:
(deriv '(x + (x + y + 2) * 3) 'x)

4

## 2.3.3 Example: Representing Sets

### Sets as unordered lists

In [57]:
(define (element-of-sets? x set)
  (cond ((null? set) #f)
        ((equal? x (car set)) #t)
        (else (element-of-sets? x (cdr set)))))

In [58]:
(define (adjoin-set x set)
  (if (element-of-sets? x set)
      set
      (cons x set)))

In [59]:
(define (intersection-set set1 set2)
  (cond ((or (null? set1) (null? set2)) '())
        ((element-of-sets? (car set1) set2)
         (cons (car set1) (intersection-set (cdr set1) set2)))
        (else (intersection-set (cdr set1) set2))))

### Exercise 2.59

In [60]:
(define (union-set set1 set2)
  (cond ((null? set1) set2)
        ((not (element-of-sets? (car set1) set2))
         (cons (car set1) (union-set (cdr set1) set2)))
        (else (union-set (cdr set1) set2))))

### Exercise 2.60

In [61]:
(define (element-of-sets2? x set)
  (cond ((null? set) #f)
        ((equal? x (car set)) #t)
        (else (element-of-sets? x (cdr set)))))
; 探すのは一般的に遅くなる。

In [62]:
(define (adjoin-set2 x set)
  (cons x set))
; ただ cons すれば良いので早くなる。

In [63]:
(define (intersection-set2 set1 set2)
  (cond ((or (null? set1) (null? set2)) '())
        ((element-of-sets2? (car set1) set2)
         (cons (car set1) (intersection-set2 (cdr set1) set2)))
        (else (intersection-set2 (cdr set1) set2))))

; コード自体は変更しなくて良い。 ele~ts2? の実行時間が遅くなったので、これも遅くなる。

In [64]:
(define (union-set2 set1 set2)
  (append set1 set2))

; append するだけ。早くなる。

追加する事に関しては早くなる。（応用例はおもいつかない。）

### Sets as ordered lists

In [65]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (car set)) #t)
        ((< x (car set)) #f)
        (else (element-of-set? x (cdr set)))))



(define (intersection-set set1 set2)
  (if (or (null? set1) (null? set2))
      '()
      (let ((x1 (car set1)) (x2 (car set2)))
        (cond ((= x1 x2)
               (cons x1 (intersection-set (cdr set1) (cdr set2))))
              ((< x1 x2)
               (intersection-set (cdr set1) set2))
              ((< x2 x1)
               (intersection-set set1 (cdr set2)))))))

### Exercise 2.61

In [66]:
(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((= x (car set)) set)
        ((< x (car set)) (cons x set))
        ((> x (car set)) (cons (car set) (adjoin-set x (cdr set))))))

### Exercise 2.62

In [67]:
(define (union-set set1 set2)
  (cond ((null? set1) set2)
        ((null? set2) set1)
        ((< (car set1) (car set2)) (cons (car set1) (union-set (cdr set1) set2)))
        ((> (car set1) (car set2)) (cons (car set2) (union-set set1 (cdr set2))))
        ((= (car set1) (car set2)) (cons (car set1) (union-set (cdr set1) (cdr set2))))))

### Sets as binary trees.

In [68]:
(define (entry tree) (car tree))
(define (left-branch tree) (cadr tree))
(define (right-branch tree) (caddr tree))
(define (make-tree entry left right)
  (list entry left right))

(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (entry set)) #t)
        ((< x (entry set)) 
         (element-of-set? x (left-branch set)))
        ((> x (entry set))
         (element-of-set? x (right-branch set)))))

(define (adjoin-set x set)
  (cond ((null? set) (make-tree x '() '()))
        ((= x (entry set)) set)
        ((< x (entry set))
         (make-tree (entry set)
                    (adjoin-set x (left-branch set))
                    (right-branch set)))
        ((> x (entry set))
         (make-tree (entry set) (left-branch set)
                    (adjoin-set x (right-branch set))))))


In [69]:
(define T0 (make-tree 2 (make-tree 1 '() '())
                        (make-tree 3 '() '())))
(print T0)

(define T1 (make-tree 7 
                      (make-tree 3 (make-tree 1 '() '()) (make-tree 5 '() '())) 
                      (make-tree 9 '() (make-tree 11 '() '()))))
(print T1)

(define T2 (make-tree 3 (make-tree 1 '() '())
                        (make-tree 7 (make-tree 5 '() '())
                                     (make-tree 9 '() (make-tree 11 '() '())))))
(print T2)

(define T3 (make-tree 5 (make-tree 3 (make-tree 1 '() '()) '())
                        (make-tree 9 (make-tree 7 '() '()) (make-tree 11 '() '()))))
(print T3)

(define T4 '())
(define T4 (adjoin-set 1 T4))
(define T4 (adjoin-set 2 T4))
(define T4 (adjoin-set 3 T4))
(define T4 (adjoin-set 4 T4))
(define T4 (adjoin-set 5 T4))
(define T4 (adjoin-set 6 T4))
(define T4 (adjoin-set 7 T4))
(print T4)

(2 (1 () ()) (3 () ()))
(7 (3 (1 () ()) (5 () ())) (9 () (11 () ())))
(3 (1 () ()) (7 (5 () ()) (9 () (11 () ()))))
(5 (3 (1 () ()) ()) (9 (7 () ()) (11 () ())))
(1 () (2 () (3 () (4 () (5 () (6 () (7 () ())))))))


### Exercise 2.63

In [70]:
(define (tree->list_1 tree)
  (if (null? tree)
      '()
      (append (tree->list_1 (left-branch tree))
              (cons (entry tree)
                    (tree->list_1
                      (right-branch tree))))))

(define (tree->list_2 tree)
  (define (copy-to-list tree result-list)
    (if (null? tree)
        result-list
        (copy-to-list (left-branch tree)
                      (cons (entry tree)
                            (copy-to-list
                             (right-branch tree)
                             result-list)))))
  (copy-to-list tree '()))

In [71]:
(print (tree->list_1 T0))
(print (tree->list_1 T1))
(print (tree->list_1 T2))
(print (tree->list_1 T3))
(print (tree->list_1 T4))

(1 2 3)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 2 3 4 5 6 7)


In [72]:
(print (tree->list_2 T0))
(print (tree->list_2 T1))
(print (tree->list_2 T2))
(print (tree->list_2 T3))
(print (tree->list_2 T4))

(1 2 3)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 2 3 4 5 6 7)


a. 見ての通り同じ結果を返す  
b. ????????????????

### Exercise 2.64

In [73]:
(define (list->tree elements)
  (car (partial-tree elements (length elements))))

(define (partial-tree elts n)
  (if (= n 0)
      (cons '() elts)
      (let ((left-size (quotient (- n 1) 2))) ; quotient : division in Z.
        (let ((left-result
               (partial-tree elts left-size)))
          (let ((left-tree (car left-result))
                (non-left-elts (cdr left-result))
                (right-size (- n (+ left-size 1))))
              (let ((this-entry (car non-left-elts))
                    (right-result
                     (partial-tree
                      (cdr non-left-elts)
                      right-size)))
                (let ((right-tree (car right-result))
                      (remaining-elts
                       (cdr right-result)))
                  (cons (make-tree this-entry
                                   left-tree
                                   right-tree)
                        remaining-elts))))))))

In [74]:
(define lst1 (list 1 3 5 7 9 11))
(print (list->tree lst1))

(define lst2 (list 3 9 5 7 1 11))
(print (list->tree lst2))


(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))
(5 (3 () (9 () ())) (1 (7 () ()) (11 () ())))


a.  
b.

In [75]:
(define lst (list 11 9 7 5 3 1 2))
(print (list->tree lst))
(print (tree->list_1 (list->tree lst)))


(5 (9 (11 () ()) (7 () ())) (1 (3 () ()) (2 () ())))
(11 9 7 5 3 1 2)


In [76]:
(define lst1 (list 1 3 7 5 3 1))
(tree->list_1 (list->tree lst1))

(1 3 7 5 3 1)

### Exercise 2.65

In [77]:
;define (intersection-set set1 set2)
; (cond ((or (null? set1) (null? set2)) '())
;       ((element-of-sets? (car set1) set2)
;         (cons (car set1) (intersection-set (cdr set1) set2)))
;        (else (intersection-set (cdr set1) set2))))
;  
;(define (union-set set1 set2)
;  (cond ((null? set1) set2)
;        ((null? set2) set1)
;        ((< (car set1) (car set2)) (cons (car set1) (union-set (cdr set1) set2)))
;        ((> (car set1) (car set2)) (cons (car set2) (union-set set1 (cdr set2))))
;        ((= (car set1) (car set2)) (cons (car set1) (union-set (cdr set1) (cdr st2))))))


;(define (adjoin-set x set)
;  (cond ((null? set) (make-tree x '() '()))
;        ((= x (entry set)) set)
;        ((< x (entry set))
;         (make-tree (entry set)
;                    (adjoin-set x (left-branch set))
;                    (right-branch set)))
;        ((> x (entry set))
;         (make-tree (entry set) (left-branch set)
;                    (adjoin-set x (right-branch set))))))



(define (union-seT t1 t2)
  (let ((s1 (tree->list_1 t1))
        (s2 (tree->list_1 t2)))
    (let ((s (union-set s1 s2)))
      (list->tree s))))


(define (intersection-seT t1 t2)
  (let ((s1 (tree->list_1 t1))
        (s2 (tree->list_1 t2)))
    (let ((s (intersection-set s1 s2)))
      (list->tree s))))

In [78]:
(intersection-seT T3 T4)

(3 (1 () ()) (5 () (7 () ())))

In [79]:
(union-seT T3 T4)

(5 (2 (1 () ()) (3 () (4 () ()))) (7 (6 () ()) (9 () (11 () ()))))

### Sets and information retrieval